In [ ]:
!pip install datasets
!pip install peft
!pip install accelerate -U

In [3]:
import transformers
from transformers import AutoTokenizer,AutoModelForCausalLM,Trainer,TrainingArguments,DataCollatorForSeq2Seq
import torch
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import json
import accelerate
from torch.utils.data import Dataset
from typing import Dict

In [4]:
transformers.__version__,accelerate.__version__

('4.39.3', '0.28.0')

In [ ]:
train_path='/kaggle/input/ruozhiba/ruozhiba.jsonl'
lora_path = '/kaggle/working/lora'
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer=AutoTokenizer.from_pretrained('Qwen/Qwen1.5-1.8B-Chat',padding_side="right",trust_remote_code=True,padding=True,truncation=True)
args = TrainingArguments(
    output_dir=lora_path,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=10,
    logging_steps=24,
    num_train_epochs=10,
    save_steps=1000,
    learning_rate=1e-3,
)

In [6]:
def process_func(example):
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
'''<|im_start|>system
请尽最大可能回答问题。<|im_end|>
<|im_start|>user
%s<|im_end|>
<|im_start|>assistant\n'''%(example['instruction']))
    response = tokenizer(f"{example['output']}")
    input_ids = instruction["input_ids"]+response["input_ids"]+[tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"]+response["attention_mask"]+[1]
    labels =[-100]*len(instruction["input_ids"])+response["input_ids"]+[tokenizer.pad_token_id]  
    return {
        "input_ids": torch.Tensor(input_ids).long().to(device),
        "attention_mask": torch.Tensor(attention_mask).long().to(device),
        "labels": torch.Tensor(labels).long().to(device)
    }
class MyDataset(Dataset):
    def __init__(self,path):
        super(MyDataset, self).__init__()
        self.data=[]
        with open(path,'r') as f:
            for i in f:
              now=json.loads(i)
              ans=process_func(now)
              self.data.append(ans)
    def __len__(self):
        return len(self.data)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(
            input_ids=self.data[i]['input_ids'],
            labels=self.data[i]['labels'],
            attention_mask=self.data[i]['attention_mask'],
        )

In [7]:
dataset=dict(train_dataset=MyDataset(train_path),eval_dataset=None)

In [8]:
print(type(dataset['train_dataset'][100]['input_ids']))

<class 'torch.Tensor'>


In [9]:
model=AutoModelForCausalLM.from_pretrained('Qwen/Qwen1.5-1.8B-Chat',torch_dtype=torch.float32).to(device)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [10]:
peft_config=LoraConfig(
         r=16,
         lora_alpha=16,
         target_modules=["q_proj","k_proj"],
         lora_dropout=0.05,
         bias="none",
         task_type="CAUSAL_LM")
lora_model=get_peft_model(model,peft_config)

In [11]:
def print_trainable_param(model):
  trainable=0
  all=0
  for _,param in model.named_parameters():
    all+=param.numel()
    if param.requires_grad:
      trainable+=param.numel()
  print(f"all:{all}||train{trainable}||train%{100*trainable/all}")
print_trainable_param(model)

all:1839974400||train3145728||train%0.17096585691627014


In [45]:
trainer = Trainer(
    model=lora_model,
    args=args,
    tokenizer=tokenizer,
    **dataset
)

In [66]:
trainer.train()

Step,Training Loss
24,0.119600
48,0.117800
72,0.118400
96,0.088100
120,0.059000
144,0.039800
168,0.028600


KeyboardInterrupt: 

In [70]:
tokenizer.padding_side='left'
model_inputs = tokenizer(['''<|im_start|>system
请尽最大可能回答问题。<|im_end|>
<|im_start|>user
老师说提一分干掉千人，那我干掉千人是不是就相当于提了一分？<|im_end|>
<|im_start|>assistant\n'''], return_tensors="pt",padding=True).to(device)
generated_ids=lora_model.generate(model_inputs.input_ids,max_new_tokens=512)

In [15]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids,generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(response)

['不一定。虽然“提一分干掉千人”意味着老师要求学生在短时间内取得一定的成绩或者进步，但是提的分数通常是指学生的最终得分或考试分数，而不仅仅是他们在某一方面的表现。\n\n例如，如果一个学生在数学考试中得了80分，这并不等于他在所有科目的考试中都得到了相同的平均分。他可能会在某些科目中表现得更好（比如他的英语考试成绩可能是全班最高的），而在其他科目中的表现可能较差。因此，“提一分干掉千人”实际上代表了他在某个特定领域的总分或平均分达到了很高的水平，而不是在整个考试成绩序列中取得了相同的成绩。\n\n另外，即使一个人在某一科目的成绩比其他人高，但在其他科目上的表现可能仍然不如人意。例如，在数学课程中，一个学生在一次重要考试中的优秀成绩并不能保证他在其他学科中的高排名。相反，他需要在其他科目中保持优异的表现，同时也要通过各种方式提高自己的学习能力和适应性。\n\n综上所述，即使一个人在“提一分干掉千人”的情况下取得了较高的分数，也不等于他们获得了与“千人”相同的整体成就，因为他们的最终成绩和表现是基于多种因素的综合评价。']


In [71]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids,generated_ids)
]
response = tokenizer.batch_decode(generated_ids)
print(response)

['这是我们上学时经常能听到的一句话，意思是在高考等重要考试中，每提高一分，就能超过很多人。这是因为考试成绩通常呈现出正态分布，所以分数越高，超越的人数就越多。这个说法并不是字面意义上的“干掉”，而是指在竞争激烈的考试中超过其他考生。\n\n因此，如果你在考试中提高了一分，确实意味着在成绩排名上超过了很多人，但这并不是实际上的“干掉”或伤害别人，而只是在分数上超越了他们。这句话强调的是在激烈的学术竞争中，即使是很小的分数提升也可能意味着在排名上的大幅跃升。<|endoftext|>']
